In [2]:
import onnx_graphsurgeon as gs
import onnx
import numpy as np
import sys
sys.path.append("..")
import onnx_utils

In [3]:
graph = gs.import_onnx(onnx.load("/home/yssun/onnx/enode/test_model/model.onnx"))

In [4]:
addNode = graph.nodes[0]

In [5]:
addNode.inputs

[Variable (input): (shape=[1024, 22], dtype=int64),
 Constant (onnx_graphsurgeon_constant_0): (shape=[1024, 22], dtype=int64)
 LazyValues (shape=[1024, 22], dtype=int64)]

In [26]:
addNode.inputs[1]

Constant (onnx_graphsurgeon_constant_0): (shape=[1024, 22], dtype=int64)
LazyValues (shape=[1024, 22], dtype=int64)

In [8]:
np_array = addNode.inputs[1].values

In [9]:
np_array.shape

(1024, 22)

In [10]:
np_array[0,:]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [12]:
np_array[slice(0,1),:]

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
sliceList = [slice(0,0),slice()]

In [5]:
input  = addNode.inputs[0]

In [6]:
reGather = graph.gather('reGather',input,np.array([0]))

In [7]:
reSlice = graph.slice('reSlice',reGather,0,12,0)

In [8]:
reUnsqueeze = graph.unsqueeze('reUnsqueeze',reSlice,[0])

In [9]:
reAdd = graph.op_with_const('Add','reAdd',reUnsqueeze,np.ones((1,12),dtype=np.int64))

In [10]:
unSlice = graph.slice('unSlice',input,12,22,1)
unAdd = graph.op_with_const('Add','unAdd',unSlice,np.ones((1024,10),dtype=np.int64))

In [11]:
@gs.Graph.register()
def tile(self,input,repeat):
  return self.layer(op="Tile",inputs=[input,repeat],outputs=["tile_out_gs"])

In [12]:
reTile = graph.tile(*reAdd,np.array([1024,1]))

In [13]:
@gs.Graph.register()
def concat(self, inputs, axis=0):
    return self.layer(
        op="Concat", inputs=inputs, attrs={"axis": axis}, outputs=["concat_out_gs"]
    )[0]

In [14]:
concatOp = graph.concat([*reTile,*unAdd],1)

In [15]:
graph.outputs.append(concatOp)

In [16]:
for out in graph.outputs:
    out.dtype = np.int64

In [17]:
model = gs.export_onnx(graph)
onnx.save(model, "remodel.onnx")

In [22]:
input1 = np.random.randint(0,100,(1024,22))

In [26]:
input1[:,:12] = input1[0,:12]

In [27]:
input1

array([[ 9, 50, 68, ..., 89, 98,  2],
       [ 9, 50, 68, ..., 72,  7, 91],
       [ 9, 50, 68, ..., 69, 10, 96],
       ...,
       [ 9, 50, 68, ..., 71,  1,  8],
       [ 9, 50, 68, ..., 42, 13, 78],
       [ 9, 50, 68, ..., 89, 89, 51]])

In [28]:
from onnxruntime import SessionOptions,ExecutionMode
import onnxruntime as ort
sess_opt = SessionOptions()
sess_opt.execution_mode  = ExecutionMode.ORT_PARALLEL
# sess_opt.inter_op_num_threads = 3
session = ort.InferenceSession("/home/yssun/onnx/enode/remodel.onnx",sess_options = sess_opt,providers=[ 'CPUExecutionProvider'])
for i in range(10):
  # random_input1 = np.random.rand(1024,22).astype(np.int64)
  random_input1 =np.random.randint(0,100,(1024,22)).astype(np.int64)
  out = session.run([],{"input":input1})

In [29]:
out

[array([[10, 51, 69, ..., 90, 99,  3],
        [10, 51, 69, ..., 73,  8, 92],
        [10, 51, 69, ..., 70, 11, 97],
        ...,
        [10, 51, 69, ..., 72,  2,  9],
        [10, 51, 69, ..., 43, 14, 79],
        [10, 51, 69, ..., 90, 90, 52]], dtype=int64),
 array([[10, 51, 69, ..., 90, 99,  3],
        [10, 51, 69, ..., 73,  8, 92],
        [10, 51, 69, ..., 70, 11, 97],
        ...,
        [10, 51, 69, ..., 72,  2,  9],
        [10, 51, 69, ..., 43, 14, 79],
        [10, 51, 69, ..., 90, 90, 52]], dtype=int64)]

In [30]:
out[0] == out[1]

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

# using function

In [66]:
# unSlice = graph.slice('unSlice',input,12,22,1)
# unAdd = graph.op_with_const('Add','unAdd',unSlice,np.ones((1024,10),dtype=np.int64))
@gs.Graph.register()
def unRedencyPartOfAdd(self,input,start,end,axes,addData,name):
  sliceNode = self.slice(name+'/unSlice',input,start,end,axes)
  unAdd = self.op_with_const('Add',name+'/unAdd',*sliceNode,addData)
  return unAdd
@gs.Graph.register()
def redencyPartofAdd(self,input,start,end,axes,addData,repeat,name,GatherAxes = [0]):
  reGather = self.gather(name+'/reGather',input,np.array(GatherAxes))
  reSlice = self.slice(name+'/reSlice',*reGather,start,end,axes)
  reUnsqueeze = self.unsqueeze(name+'/reUnsqueeze',*reSlice,[0])
  reAdd = self.op_with_const('Add',name+'/reAdd',*reUnsqueeze,addData)
  reTile = self.tile(*reAdd,np.array(repeat))
  return reTile

@gs.Graph.register()
def addRedencyRewrite(self,input,name,
                      unRedencyStart,unRedencyEnd,unRedencyAxes,unRedencyAddData,
                      redencyStart,redencyEnd,redencyAxes,redencyRepeat,redencyAddData,redencyGatherAxes):
  unRedencyPart = self.unRedencyPartOfAdd(input,unRedencyStart,unRedencyEnd,unRedencyAxes,unRedencyAddData,name)
  redencyPart = self.redencyPartofAdd(input,redencyStart,redencyEnd,redencyAxes,redencyAddData,redencyRepeat,name,redencyGatherAxes)
  concatOp = self.concat([*redencyPart,*unRedencyPart],1)
  return concatOp






In [67]:
graph1 = gs.import_onnx(onnx.load("/home/yssun/onnx/enode/test_model/model.onnx"))

In [68]:
reOp = graph1.addRedencyRewrite(graph1.nodes[0].inputs[0],'/test',
                                12,22,1,np.ones((1024,10),dtype=np.int64),0,12,0,np.array([1024,1]),np.ones((1,12),dtype=np.int64),[0])

In [69]:
graph1.outputs=[reOp]

In [70]:
graph1.nodes = graph1.nodes[1:]

In [71]:
for out in graph1.outputs:
    out.dtype = np.int64

In [72]:
graph.cleanup().toposort()


[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 13. 
[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[E] Function: '_ipython_display_' was not registered for opset 13. 
[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 13. 
[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[E] Function: '_repr_mimebundle_' was not registered for opset 13. 
[W] colored module is not installed, will not use colors when 

Graph onnx_graphsurgeon_graph (Opset 13)
Local Functions: []
Inputs: [Variable (input): (shape=[1024, 22], dtype=int64)]
Nodes: onnx_graphsurgeon_node_2 (Add)
	Inputs: [
		Variable (input): (shape=[1024, 22], dtype=int64)
		Constant (onnx_graphsurgeon_constant_0): (shape=(1024, 22), dtype=int64)
	]
	Outputs: [
		Variable (add_out_gs_1): (shape=[1024, 22], dtype=int64)
	]
Outputs: [Variable (add_out_gs_1): (shape=[1024, 22], dtype=int64)]

In [73]:
model = gs.export_onnx(graph1)
onnx.save(model, "remodel1.onnx")

In [74]:
from onnxruntime import SessionOptions,ExecutionMode
import onnxruntime as ort
sess_opt = SessionOptions()
sess_opt.execution_mode  = ExecutionMode.ORT_PARALLEL
# sess_opt.inter_op_num_threads = 3
session = ort.InferenceSession("/home/yssun/onnx/enode/test_book/remodel1.onnx",sess_options = sess_opt,providers=[ 'CPUExecutionProvider'])
for i in range(10):
  random_input1 = np.random.rand(1024,22).astype(np.int64)
  out = session.run([],{"input":random_input1})